<a href="https://colab.research.google.com/github/KPraveenRaj/IPCV_PROJECT-Implementation/blob/main/IPCV_Project_PFT_SR_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Edit the drive path here

In [1]:
drive_cache = '/content/drive/MyDrive/Colab_Notebooks/MTech_IPCV/IPCV_Project/PFT_SR_CACHE'

In [2]:
import os
from google.colab import files
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Create cache directories in Drive
os.makedirs(drive_cache, exist_ok=True)
os.makedirs(drive_cache + '/repo', exist_ok=True)
os.makedirs(drive_cache + '/cuda_ext', exist_ok=True)
print('✅ Drive cache ready:', drive_cache)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive cache ready: /content/drive/MyDrive/Colab_Notebooks/MTech_IPCV/IPCV_Project/PFT_SR_CACHE


In [3]:
cache_repo = drive_cache + '/repo'

if not os.listdir(cache_repo):
    print('Cloning repo first time...')
    !git clone https://github.com/LabShuHangGU/PFT-SR.git
    !cp -r /content/PFT-SR/* {cache_repo}/
else:
    print('Loading repo from Drive cache...')
    !cp -r {cache_repo} /content/
    !mv /content/repo /content/PFT-SR

%cd /content/PFT-SR

Loading repo from Drive cache...
/content/PFT-SR


# Maually load the Pretrained Models

####Load the pretrained models into "./experiments/pretrained_models" from [here](https://drive.google.com/drive/folders/1ChkxVDghFWUtJydJKLp5yssrUfm0VWfg)

(if you have already loaded the models in the drive cache previously, it'll load automatically)

In [4]:
# Define paths. drive_cache is defined in a previous cell.
model_cache_path = os.path.join(drive_cache, 'repo/experiments/pretrained_models')
local_model_path = '/content/PFT-SR/experiments/pretrained_models'


# Ensure local model directory exists
os.makedirs(local_model_path, exist_ok=True)
# Ensure model cache directory exists in Drive
os.makedirs(model_cache_path, exist_ok=True)

# Check if models are in Drive cache
if len(os.listdir(model_cache_path))>1:
    print("Pretrained models found in Drive cache. Must be already loaded to local PFT-SR directory...")
    print(os.listdir(model_cache_path))
    print("✅ Models loaded from cache.")
else:
    print("Pretrained models not found in Drive cache. Please upload the models to continue.")
    print("You can find them at: https://drive.google.com/drive/folders/1ChkxVDghFWUtJydJKLp5yssrUfm0VWfg")
    uploaded = files.upload()

    if uploaded:
        print("Saving uploaded models to Drive cache and local PFT-SR directory...")
        for filename in uploaded.keys():
            # Move uploaded files to local model path
            !mv {filename} {local_model_path}/
            # Also copy to Drive cache for future sessions
            !cp {local_model_path}/{filename} {model_cache_path}/
        print("✅ Models uploaded and saved to cache.")
    else:
        print("🛑 No models uploaded. Please upload the necessary models manually.")


Pretrained models found in Drive cache. Must be already loaded to local PFT-SR directory...
['101_PFT_light_SRx2_scratch.pth', '103_PFT_light_SRx4_finetune.pth', '102_PFT_light_SRx3_finetune.pth', '001_PFT_SRx2_scratch.pth', '002_PFT_SRx3_finetune.pth', '003_PFT_SRx4_finetune.pth', 'README.md']
✅ Models loaded from cache.


# After the below runtime setups restart the runtime

In [5]:
# Install CUDA Toolkit 12.1 (only if cached nvcc does not exist)
if not os.path.exists('/usr/local/cuda-12.1/bin/nvcc'):
    print('Installing CUDA Toolkit 12.1...')
    !wget https://developer.download.nvidia.com/compute/cuda/12.1.0/local_installers/cuda_12.1.0_530.30.02_linux.run
    !sudo sh cuda_12.1.0_530.30.02_linux.run --silent --toolkit
else:
    print('CUDA Toolkit already installed in this session.')

os.environ['CUDA_HOME'] = '/usr/local/cuda-12.1'
os.environ['PATH'] += ':/usr/local/cuda-12.1/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.1/lib64'

!nvcc --version

CUDA Toolkit already installed in this session.
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Feb__7_19:32:13_PST_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0


## The below install will also takes time please wait
### ‼️ (IMPORTANT: Despite the warning to restart the runtime do not restart runtime yet)

### After restart run all the cell again but do not restart other than first run. (You need to do this every time the runtime is fresh/deleted)

In [6]:
# Install PyTorch 2.5.0+cu121
!pip install --upgrade --force-reinstall torch==2.5.0+cu121 torchvision==0.20.0+cu121 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.0%2Bcu121-cp312-cp312-linux_x86_64.whl (780.4 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.20.0%2Bcu121-cp312-cp312-linux_x86_64.whl (7.3 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.0%2Bcu121-cp312-cp312-linux_x86_64.whl (3.4 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none

In [7]:
import torch
print('CUDA from PyTorch:', torch.version.cuda, 'Available:', torch.cuda.is_available())

CUDA from PyTorch: 12.1 Available: True


Install the below dependencies too

In [8]:
!pip install --upgrade addict future lmdb numpy opencv-python Pillow \
        pyyaml requests scikit-image scipy tb-nightly tqdm yapf fairscale

  Using cached numpy-2.3.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pillow-12.0.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.8 kB)
  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.5 MB)
Using cached pillow-12.0.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (7.0 MB)
  Attempting uninstall: Pillow
    Found existing installation: pillow 11.3.0
    Uninstalling pillow-11.3.0:
      Successfully uninstalled pillow-11.3.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.3
    Uninstalling numpy-2.3.3:
      Successfully uninstalled numpy-2.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 req

Build the custom CUDA cache

In [9]:
# Build CUDA extension
print('Compiling CUDA extension...')
!python setup.py develop
%cd ops_smm
!bash make.sh
%cd ..

Compiling CUDA extension for the first time...
/usr/local/lib/python3.12/dist-packages/setuptools/__init__.py:80: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.12/dist-packages/setuptools/command/develop.py:42: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/s

## Run the below cell to test with a sample image

In [12]:
uploaded = files.upload()
input_image = list(uploaded.keys())[0]
print('Uploaded:', input_image)

# Get base name and extension
base_name, ext = os.path.splitext(os.path.basename(input_image))

!python inference.py -i $input_image -o results/colab_output/ --scale 4 --task lightweight

output_image = f"results/colab_output/{base_name}_PFT_lightweight_SRx4{ext}"
files.download(output_image)


Saving inference_image.png to inference_image.png
Uploaded: inference_image.png
/usr/local/lib/python3.12/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/content/PFT-SR/inference.py:87: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unles

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# For batch images make a directory and mention it.
### "!python inference.py -i "directory" -o results/test/ --scale 4 --task lightweight"

In [13]:
!python inference.py -i inference_images/ -o results/test/ --scale 4 --task lightweight

/usr/local/lib/python3.12/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/content/PFT-SR/inference.py:87: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_